In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import time
from tqdm import tqdm
import os
from glob import glob
import pathlib
import geopandas as gpd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch

import rasterio
import rasterio.plot
from rasterio.features import rasterize
import random
import shapely
import functools

from collections import defaultdict
import io

In [ ]:
powiaty_fname = '/mnt/nfs/ortofotomapa/shp/powiaty/powiaty.shp'
powiaty = gpd.read_file(powiaty_fname)
powiaty

In [ ]:
powiaty.info()

In [ ]:
powiaty.iloc[0]

In [ ]:
powiaty.iloc[0].geometry

In [ ]:
powiaty.iloc[0].geometry.exterior

In [ ]:
ext = np.array(powiaty.iloc[0].geometry.exterior.xy)
print(ext.shape)
print(np.min(ext,axis=1))
print(np.max(ext,axis=1))
print(np.mean(ext,axis=1))

In [ ]:
plt.figure(figsize=(20,20))
for i,p in enumerate(powiaty.iloc):
    if 'MultiPolygon' in str(type(p.geometry)):
        print(i, p.JPT_NAZWA_)
        ext = np.array(p.geometry[0].exterior.xy)
    else:
        ext = np.array(p.geometry.exterior.xy)
#     print(np.mean(ext,axis=1))
    plt.plot(*np.mean(ext,axis=1), marker='x', label=p.JPT_NAZWA_)
#     break
# plt.legend()

In [ ]:
powiaty.iloc[12].geometry

In [ ]:
powiaty.geometry.plot(figsize=(20,20))

In [ ]:
powiaty.JPT_KOD_JE.unique()

In [ ]:
len(sorted(powiaty.JPT_KOD_JE.unique()))

# The dataset

In [ ]:
!ls

In [ ]:
class BDOT10kDataset(torch.utils.data.Dataset):
    """BDOT10k dataset"""
    def __init__(self,
                 tiff_dir='/mnt/nfs/ortofotomapa/images',
                 shp_dir='/mnt/nfs/ortofotomapa/shp/unzipped/SHP/',
                 powiaty_shp_fname='/mnt/nfs/ortofotomapa/shp/powiaty/powiaty.shp',
                 bdot10k_cats_fname='../data/BDOT10k-categories.csv',
                 code='BUBD',
                 size=1024,
                 transform=None):
        self.tiff_dir = tiff_dir
        self.tiffs = sorted(glob(tiff_dir+'/*.tif'))
        print(f'Found {len(self.tiffs)} tiff files in {self.tiff_dir}')
        self.powiaty = gpd.read_file(powiaty_shp_fname)
        print(f'Found {len(self.powiaty)} powiats')
        
        self.shps = sorted(glob(shp_dir+'*.shp'))
        print(f'SHPs: {len(self.shps)}')
        
        
        self.bdot10k_df = pd.read_csv(bdot10k_cats_fname)
        self.bdot10k_cats = sorted(self.bdot10k_df.Kod2.tolist())
        self.bdot10k_cats_dict = defaultdict(list)
        
        for k in self.bdot10k_cats:
            self.bdot10k_cats_dict[k[:2]].append(k)
        
        for k in self.bdot10k_cats_dict:
            print(f'{k}:',self.bdot10k_cats_dict[k])
#             for c in self.bdot10k_cats_dict[k]:
#                 print(f'- {c}')


        self.code = code
        self.transform = transform
        self.size = size

    def __len__(self):
        return len(self.tiffs)

    def get_tile(self, raster):
        img = raster.read().transpose(1,2,0)
#         print(img.shape)
#         raster.transform * (0,0), raster.transform * (raster.width, raster.height)
        x0 = random.randint(0,img.shape[1]-self.size-1)
        y0 = random.randint(0,img.shape[0]-self.size-1)
        x1 = x0+self.size
        y1 = y0+self.size
        
#         print(x0,x1,y0,y1)
        _x0, _y0 = raster.transform * (x0,y0)
        _x1, _y1 = raster.transform * (x1,y1)
        
        _x0,_x1 = min(_x0,_x1), max(_x0,_x1)
        _y0,_y1 = min(_y0,_y1), max(_y0,_y1)
        
        return img[y0:y1,x0:x1], (_x0,_y0), (_x1,_y1)
    
    def get_id_by_code(self, code):
        return self.bdot10k_cats.index(code)
    
    def get_multiple_ids(self, code):
        return self.bdot10k_cats.index(code)
    
#     @functools.lru_cache(maxsize=10)
    def load_tiff(self, idx):
        tiff_fname = self.tiffs[idx]
#         print('loading:',tiff_fname)
        raster = rasterio.open(tiff_fname)
        return raster
    
    def get_image(self, idx):
        raster = self.load_tiff(idx)
        plt.figure(figsize=(20,20))
#         rasterio.plot.show(raster)
#         plt.show()
        img, (x0,y0),(x1,y1) = self.get_tile(raster)
        return img, x0,y0,x1,y1
        
    def get_heads_count(self):
        return len(ds.bdot10k_cats_dict.keys())

    def get_label(self, df):
        mask = np.zeros((self.get_heads_count(), self.size, self.size), dtype='int')
        if df is not None:
            for code in df.X_KOD.unique():
                out_id = cat_names.index(code[:2])
        #         code_int = int(code[4:])
                code_int = ds.get_id_by_code(code)
                m = code_int * rasterize(shapes=df[df.X_KOD==code].geometry.iloc,
                         out_shape=(self.size, self.size))[::-1]
                mask[out_id][m!=0] = m[m!=0]
        return mask
        
    @functools.lru_cache(maxsize=100)
    def load_bdot10k_for_powiat(self, powiat_id):
        df_to_merge = []
        for fn in self.shps:
            if self.code in fn:
                    if powiat_id in fn:
#                         print(powiat_id, fn)
                        df = gpd.read_file(fn)
                        df_to_merge.append(df)
        if len(df_to_merge)>0:
            df_merged = pd.concat(df_to_merge)
            return df_merged
        else:
            return None
    
    def load_bdot10k_for_powiats(self, powiat_ids):
        df_to_merge = []
        for powiat_id in powiat_ids:
            df = self.load_bdot10k_for_powiat(powiat_id)
            df_to_merge.append(df)
        if len(df_to_merge)>0:
            df_merged = pd.concat(df_to_merge)
            return df_merged
        else:
            return None
    
    def powiat_ids_for_bbox(self,x0,y0,x1,y1):
        bbox = shapely.geometry.box(x0,y0,x1,y1)

        powiat_ids = []
        for p in powiaty.iloc:
            if p.geometry.contains(bbox) or bbox.contains(p.geometry):
                powiat_ids.append(p.JPT_KOD_JE)
        return powiat_ids
    
    def plot_sample(self, img, mask, show=False):
        fig = plt.figure(figsize=(20,10))
        plt.subplot(2,5,1)
        plt.imshow(img)
    #     plt.axis('off')
        for i,vmax in enumerate([len(_) for _ in ds.bdot10k_cats_dict.values()]):
            plt.subplot(2,5,2+i)
    #         plt.imshow(mask[i],cmap='hsv')
            plt.imshow(mask[i],cmap='gist_ncar',vmin=0, vmax=vmax)
            plt.axis('off')
            plt.title(cat_names[i]+'\n'+ ds.bdot10k_df[ds.bdot10k_df.Kod0==cat_names[i]].iloc[0]['Nazwa kategorii klas obiektów'])
        plt.tight_layout()
        if show:
            plt.show()
        else:
            with io.BytesIO() as buff:
                fig.savefig(buff, format='raw')
                buff.seek(0)
                data = np.frombuffer(buff.getvalue(), dtype=np.uint8)
            w, h = fig.canvas.get_width_height()
            img = data.reshape((int(h), int(w), -1))
            return img
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img, x0,y0,x1,y1 = self.get_image(idx)
        
#         plt.figure(figsize=(10,10))
#         plt.imshow(img)
        
        powiat_ids = self.powiat_ids_for_bbox(x0,y0,x1,y1)
#         print(x0,y0,'-->',x1,y1,powiat_ids)
        
        
        df = self.load_bdot10k_for_powiats(powiat_ids)
        
#         df.geometry.plot(figsize=(10,10))
#         plt.show()
#         df.geometry.plot(figsize=(10,10))
#         plt.xlim(x0,x1)
#         plt.ylim(y0,y1)
#         plt.show()
#         print('all:',len(df))
        if df is not None:
            df = df.cx[x0:x1, y0:y1]
            
            new_geoms = []
            for i in range(len(df)):
                s = df.iloc[i].geometry
                if 'MultiPolygon' in str(type(df.iloc[i].geometry)):
                    ps = []
                    for p in df.iloc[i].geometry:
                        xys = np.array([_ for _ in zip(*p.exterior.xy)])-(x0, y0)
                        xys[:,0]=xys[:,0]*img.shape[1]/(x1-x0)
                        xys[:,1]=xys[:,1]*img.shape[0]/(y1-y0)
                        ps.append(shapely.geometry.Polygon(xys))
                    s = shapely.geometry.MultiPolygon(ps)
                elif 'Polygon' in str(type(df.iloc[i].geometry)):
                    xys = np.array([_ for _ in zip(*df.iloc[i].geometry.exterior.xy)])-(x0, y0)
                    xys[:,0]=xys[:,0]*img.shape[1]/(x1-x0)
                    xys[:,1]=xys[:,1]*img.shape[0]/(y1-y0)
                    s = shapely.geometry.Polygon(xys)
                elif 'LineString' in str(type(df.iloc[i].geometry)):
                    xys = np.array(df.iloc[i].geometry.coords)-(x0, y0)
                    xys[:,0]=xys[:,0]*img.shape[1]/(x1-x0)
                    xys[:,1]=xys[:,1]*img.shape[0]/(y1-y0)
                    s = shapely.geometry.LineString(xys)
                elif 'MultiPoint' in str(type(df.iloc[i].geometry)):
                    ps = []
                    for p in df.iloc[i].geometry.geoms:
                        x = (p.x-x0)*img.shape[1]/(x1-x0)
                        y = (p.y-y0)*img.shape[0]/(y1-y0)
                        ps.append((x,y))
                    s = shapely.geometry.MultiPoint(ps)
                elif 'Point' in str(type(df.iloc[i].geometry)):
                    p = df.iloc[i].geometry
                    x = (p.x-x0)*img.shape[1]/(x1-x0)
                    y = (p.y-y0)*img.shape[0]/(y1-y0)
                    s = shapely.geometry.Point(x,y)
                else:
                    print(type(df.iloc[i].geometry))
    #                 print(df.iloc[i])
                new_geoms.append(s)

            df.geometry = new_geoms
        else:
            df = None
                   
        label = self.get_label(df)
#         print('cx:',len(df))
        
#         df.geometry.plot(figsize=(10,10))
#         plt.show()
#         df.geometry.plot(figsize=(10,10))
#         plt.xlim(x0,x1)
#         plt.ylim(y0,y1)
#         plt.show()
        
        
        return img, label

#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
#         sample = {'image': image, 'landmarks': landmarks}

#         if self.transform:
#             sample = self.transform(sample)

#         return sample

ds = BDOT10kDataset(size=2048, code='_')

In [ ]:
cat_names = list(ds.bdot10k_cats_dict.keys())
for ii,b in enumerate(ds):
    print(ii)
    img, mask = b
    print(img.shape, mask.shape)
    ds.plot_sample(img,mask, )
#     plt.figure(figsize=(20,10))
#     plt.subplot(2,5,1)
#     plt.imshow(img)
# #     plt.axis('off')
#     for i,vmax in enumerate([len(_) for _ in ds.bdot10k_cats_dict.values()]):
#         plt.subplot(2,5,2+i)
# #         plt.imshow(mask[i],cmap='hsv')
#         plt.imshow(mask[i],cmap='gist_ncar',vmin=0, vmax=vmax)
#         plt.axis('off')
#         plt.title(cat_names[i]+'\n'+ ds.bdot10k_df[ds.bdot10k_df.Kod0==cat_names[i]].iloc[0]['Nazwa kategorii klas obiektów'])
#     plt.tight_layout()
#     plt.show()
#         print(i,'-->',np.unique(mask[i],return_counts=True))
    
    if ii>10:
        break

In [ ]:
res = ds.plot_sample(img,mask)
plt.figure()
plt.imshow(res)

In [ ]:
ds.plot_sample(img,mask)

In [ ]:
np.unique(mask,return_counts=True)


In [ ]:
mask = None

for code in df.X_KOD.unique():
    code_int = int(code[4:])
    if mask is None:
        mask = code_int * rasterize(shapes=df[df.X_KOD==code].geometry.iloc,
                 out_shape=(img.shape[0],img.shape[1]))[::-1]
    else:
        m = code_int * rasterize(shapes=df[df.X_KOD==code].geometry.iloc,
                 out_shape=(img.shape[0],img.shape[1]))[::-1]
        mask[m!=0] = m[m!=0]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(mask,cmap='tab20b')
np.unique(mask,return_counts=True)